(rv)=

# Radial velocity fitting

In [223]:
import sys
sys.version

'3.9.5 | packaged by conda-forge | (default, Jun 19 2021, 00:27:35) \n[Clang 11.1.0 ]'

In [224]:
import exoplanet
print(f"exoplanet.__version__ = '{exoplanet.__version__}'")

exoplanet.__version__ = '0.5.1'


Fitting RVs for Earth and Jupiter

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import exoplanet as xo
import pymc3 as pm
import pymc3_ext as pmx
from astropy import units as u
from astropy.constants import M_earth, M_sun
from simulate_data import *

import matplotlib 
matplotlib.rc('xtick', labelsize=18) 
matplotlib.rc('ytick', labelsize=18)

T_subtract = 2454000
# orbital parameters from https://www.princeton.edu/~willman/planetary_systems/Sol/
# BJD determined by converting values above using https://ssd.jpl.nasa.gov/tc.cgi#top

P_earth = 365.256
e_earth = 0.0167
Tper_earth= 2454115.5208333 - T_subtract
omega_earth = np.radians(102.9)
Omega_earth = np.radians(0.0)
inclination_earth = np.radians(45.0)
m_earth = 1*3.00273e-6 #units m_sun



P_jup = 4327.631
e_jup = 0.0484
Tper_jup = 2455633.7215278- T_subtract
omega_jup = np.radians(274.3) - 2*np.pi
Omega_jup = np.radians(100.4)
inclination_jup = np.radians(0) + inclination_earth
m_jup = 317.83*3.00273e-6 #units m_sun




WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


In [28]:
orbit_params_earth = [P_earth, e_earth, Tper_earth, omega_earth, Omega_earth, inclination_earth, m_earth]
orbit_params_jup = [P_jup, e_jup, Tper_jup, omega_jup, Omega_jup, inclination_jup, m_jup]

orbit_params = [orbit_params_earth, orbit_params_jup]

################

orbit_params_earth2 = [P_earth, e_earth, Tper_earth, omega_earth, Omega_earth-50, inclination_earth, m_earth]
orbit_params_jup2 = [P_jup, e_jup, Tper_jup, omega_jup, Omega_jup+50, inclination_jup, m_jup]

orbit_params2 = [orbit_params_earth2, orbit_params_jup2]


times_observed_astrometry = []
t_0 = int(Tper_earth)
for ii in range(t_0, t_0+1800):
    if ii % 50 == 0:
        times_observed_astrometry.append(ii)

n_planets = 2
t_dur_rv = 3650
n_obs_rv = 3000
sigma_rv = 0.0

sigma_theta = 0.0
sigma_rho = 0.0
plx = 0.1



################

times, rv_results, theta_results, rho_results = simulate_data(
    n_planets, 
    sigma_rv, 
    sigma_theta,
    sigma_rho,
    plx,
    orbit_params,
    t_dur_rv = t_dur_rv,
    n_obs_rv = n_obs_rv,
    times_observed_astrometry = times_observed_astrometry
    )


[_,_,[theta_orbit, theta_orbit_sum, theta_sim, theta_sim_sum],_]  = \
    times, rv_results, theta_results, rho_results


################

times, rv_results, theta_results, rho_results = simulate_data(
    n_planets, 
    sigma_rv, 
    sigma_theta,
    sigma_rho,
    plx,
    orbit_params2,
    t_dur_rv = t_dur_rv,
    n_obs_rv = n_obs_rv,
    times_observed_astrometry = times_observed_astrometry
    )


[_,_,[theta_orbit2, theta_orbit_sum2, theta_sim2, theta_sim_sum2],_]  = \
    times, rv_results, theta_results, rho_results


In [29]:
theta_sim_sum.all == theta_sim_sum2.all

False

In [30]:
theta_sim_sum

array([ 0.74569799,  1.46940515,  2.24908755, -2.88006633, -1.96344569,
       -1.28922085, -0.37763475,  0.84640823,  1.64755731,  2.35476037,
       -2.87135371, -1.82002472, -1.11866391, -0.33019807,  0.89260248,
        1.83267677,  2.52636753, -2.82006866, -1.65458192, -0.865991  ,
       -0.13546809,  1.00235808,  2.12360063,  2.86768624, -2.56754051,
       -1.35319444, -0.41978466,  0.31284167,  1.32713206,  2.59533437,
       -2.85295064, -2.07140738, -0.92555908,  0.14080199,  0.87614917,
        1.73405806])

In [31]:
theta_sim_sum2

array([ 0.74569799,  1.46940515,  2.24908755, -2.88006633, -1.96344569,
       -1.28922085, -0.37763475,  0.84640823,  1.64755731,  2.35476037,
       -2.87135371, -1.82002472, -1.11866391, -0.33019807,  0.89260248,
        1.83267677,  2.52636753, -2.82006866, -1.65458192, -0.865991  ,
       -0.13546809,  1.00235808,  2.12360063,  2.86768624, -2.56754051,
       -1.35319444, -0.41978466,  0.31284167,  1.32713206,  2.59533437,
       -2.85295064, -2.07140738, -0.92555908,  0.14080199,  0.87614917,
        1.73405806])